In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

## Solo IPS con camas disponibles, y en bogota 

In [2]:
capacities = pd.read_csv('../data/01_raw/capacidad.csv')
capacities.columns = [c.strip() for c in capacities.columns.values]

In [3]:
# found a better way
#ips_geo = pd.read_csv('../data/01_raw/ips_geo.csv')

In [4]:
capacities.dtypes

nom sede IPS                         object
nit IPS                             float64
naturaleza                           object
num digito_verificion               float64
num nivel atencion                  float64
nom grupo capacidad                  object
nom descripcion capacidad            object
num cantidad capacidad instalada    float64
dtype: object

In [6]:
# select important columns and rename
capacities = capacities.iloc[:,[0,1,2,4,5,6,7]]
capacities.columns =[
    'nom_sede_ips',
    'nit_ips',
    'naturaleza',
    'nivel_atencion',
    'grupo_capacidad',
    'descripcion_capacidad',
    'capacidad']

In [7]:
capacities.head()

,nom_sede_ips,nit_ips,naturaleza,nivel_atencion,grupo_capacidad,descripcion_capacidad,capacidad
0,CLINICA VIVE,900450008.0,Privada,NaN,CAMAS,Psiquiatría,44.0
1,CAMPONUEVO CISNE 2,830100595.0,Privada,NaN,CAMAS,Psiquiatría,15.0
2,COVEN,900280825.0,Privada,NaN,CAMAS,Adultos,12.0
3,GASTROCAL SAS,900690781.0,Privada,NaN,SALAS,Procedimientos,2.0
4,DACARE IPS,900883992.0,Privada,NaN,CAMAS,Adultos,2.0


Algunas no tienen nit y lo imputamos por ahora con 0

In [9]:
capacities['nit_ips'] = capacities['nit_ips'].fillna(0).astype(int).astype(str)

<ipython-input-9-338d3c73c266>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capacities['nit_ips'] = capacities['nit_ips'].fillna(0).astype(int).astype(str)


In [10]:
# guardamos archivo intermedio
capacities.to_parquet('../data/02_processed/capacities.parquet')

In [11]:
# esto permite obtener el conjunto unico de ips. Algo asi como un id
national_ips = capacities[['nom_sede_ips', 'nit_ips']].drop_duplicates()

In [12]:
national_ips.shape

(3765, 2)

## Ubicacion de las IPS

importamos la data de ips de la informacion del catastro

In [13]:
import requests

In [14]:
# read all ips from esri query
json_url = "https://catalogopmb.catastrobogota.gov.co/PMBWeb/web/exportar?cmd=export&ID_SERVICIO=541&xmin=-8372083.210284375&ymin=475445.22783197014&xmax=-8137268.659392426&ymax=551882.2561171096&spatialReference=102100"

r = requests.get(json_url)

In [15]:
r.status_code

200

In [16]:
ips_geo = pd.DataFrame(r.json()['resultados'])

In [17]:
ips_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2984 entries, 0 to 2983
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Dirección                         2984 non-null   object
 1   Clase de Prestador                2984 non-null   object
 2   Número Identificación Tributaria  2984 non-null   object
 3   Naturaleza Jurídica               2984 non-null   object
 4   Correo Electrónico                2984 non-null   object
 5   Clase de Persona                  2984 non-null   object
 6   Código de Habilitación            2984 non-null   int64 
 7   OBJECTID                          2984 non-null   int64 
 8   Nombre de la IPS                  2984 non-null   object
 9   Object_ID                         2984 non-null   int64 
 10  Número de Fax                     2984 non-null   object
 11  Número de Teléfono                2984 non-null   object
 12  Código del Prestador

In [18]:
# select columns and rename
ips_geo = ips_geo.iloc[:, [0,1,2,6,7,8,13]]
ips_geo.columns = [
    'direccion',
    'clase_de_prestador',
    'nit_ips',
    'codigo_de_habilitacion',
    'id_ips',
    'nombre_ips',
    'geometry']


# [c.lower().replace(' ', '_') for c in ips_geo.columns]

In [19]:
ips_geo = (ips_geo.
     assign(longitud_ips=lambda x: x.geometry.apply(lambda g: g['x']),
            latitud_ips=lambda x: x.geometry.apply(lambda g: g['y'])).
     drop('geometry', axis=1)
)

ips_geo.head()

,direccion,clase_de_prestador,nit_ips,codigo_de_habilitacion,id_ips,nombre_ips,longitud_ips,latitud_ips
0,"AV. CRA. 9 116 - 20 CS. 201, 219A, 219B.",Instituciones - IPS,800173625,110010199501,2,LABORATORIO DE OTONEUROLOGIA - INSTITUTO NACIO...,-74.033780,4.694807
1,AC 33 # 14 37,Instituciones - IPS,830099212,110010905601,4,CENTRO DE INVESTIGACIONES ONCOLOGICAS CLINICA ...,-74.069838,4.621248
2,AC 13 N 65 -21,Instituciones - IPS,800149384,110010918632,9,CLINISANITAS DIAGNOSTICO ZONA IN,-74.116059,4.634265
3,CL 63 N 22-22,Instituciones - IPS,900116722,110011709301,10,MISION SALUD INTERNACIONAL SAS CALLE 63,-74.071383,4.651043
4,AUTOP N No. 95-29,Instituciones - IPS,900098985,110011960301,17,VIHONCO IPS BOGOTA,-74.058287,4.683874


In [20]:
print(ips_geo.shape)
print(ips_geo.nit_ips.nunique())

(2984, 8)
1617


## ips importantes
Vamos a asumir que las ips que requieren mejor eficiencia en el flujo son las ips que pueden atender casos del virus, y tienen camas, estas ips se encuentran especificadas en el dataset de capacidades

In [21]:
ips_geo_important = ips_geo.loc[ips_geo['nit_ips'].isin(national_ips['nit_ips'])]

In [22]:
ips_geo.shape

(2984, 8)

In [23]:
ips_geo_important

,direccion,clase_de_prestador,nit_ips,codigo_de_habilitacion,id_ips,nombre_ips,longitud_ips,latitud_ips
1,AC 33 # 14 37,Instituciones - IPS,830099212,110010905601,4,CENTRO DE INVESTIGACIONES ONCOLOGICAS CLINICA ...,-74.069838,4.621248
2,AC 13 N 65 -21,Instituciones - IPS,800149384,110010918632,9,CLINISANITAS DIAGNOSTICO ZONA IN,-74.116059,4.634265
3,CL 63 N 22-22,Instituciones - IPS,900116722,110011709301,10,MISION SALUD INTERNACIONAL SAS CALLE 63,-74.071383,4.651043
5,KR 13 B # 161-85,Instituciones - IPS,860035992,110010911103,19,FUNDACIÓN CARDIOINFANTIL-INSTITUTO DE CARDIOLOGÍA,-74.034409,4.740787
9,Carrera 19 N 102-31,Instituciones - IPS,830018305,110010919912,38,JAVESALUD ALLIANZ,-74.052213,4.688005
...,...,...,...,...,...,...,...,...
2973,Calle 57 Sur # 89B - 19,Instituciones - IPS,900959048,110013029641,2958,Unidad de Servicios de Salud Cabañas,-74.190161,4.630956
2974,Transversal 72b No. 44D - 21 Sur,Instituciones - IPS,900959048,110013029649,2962,Unidad de Servicios de Salud Nuevas Delicias,-74.146654,4.599930
2975,Cra 78 #35-21 Sur,Instituciones - IPS,900959048,110013029651,2964,Unidad de Servicios de Salud 29 Kennedy,-74.148330,4.623594
2976,CARRERA 19A N° 85-11,Instituciones - IPS,800127648,110010845605,2972,HORUS IBERIA CALLE 85,-74.057976,4.671706


In [24]:
ips_geo_important.to_csv('../data/02_processed/ips_geo_important.csv')
ips_geo_important.to_parquet('../data/02_processed/ips_geo_important.parquet')

Same with capacities, we can filter only those that ended up in important

In [25]:
capacities_important = capacities.loc[lambda x: x.nit_ips.isin(ips_geo_important.nit_ips)]

In [26]:
capacities_important.to_parquet('../data/02_processed/capacities_important.parquet')
capacities_important.to_csv('../data/02_processed/capacities_important.csv')

This data need further processing, but well see what needs to be done in `main_analysis`

## Transmilenio data

In [27]:
estaciones = pd.read_csv('../data/01_raw/estaciones_transmilenio.csv')

In [28]:
estaciones.columns

Index(['X', 'Y', 'objectid', 'numero_estacion', 'nombre_estacion',
       'coordenada_x_estacion', 'coordenada_y_estacion', 'ubicacion_estacion',
       'troncal_estacion', 'numero_vagones_estacion',
       'numero_accesos_estacion', 'biciestacion_estacion',
       'capacidad_biciestacion_estacion', 'tipo_estacion',
       'biciparqueadero_estacion', 'latitud_estacion', 'longitud_estacion',
       'globalid'],
      dtype='object')

In [29]:
estaciones = estaciones[['numero_estacion', 'nombre_estacion', 'ubicacion_estacion', 'troncal_estacion',
                         'numero_vagones_estacion',
       'numero_accesos_estacion', 'biciestacion_estacion',
       'capacidad_biciestacion_estacion', 'tipo_estacion',
       'biciparqueadero_estacion', 'latitud_estacion', 'longitud_estacion']]

In [30]:
estaciones.head()

,numero_estacion,nombre_estacion,ubicacion_estacion,troncal_estacion,numero_vagones_estacion,numero_accesos_estacion,biciestacion_estacion,capacidad_biciestacion_estacion,tipo_estacion,biciparqueadero_estacion,latitud_estacion,longitud_estacion
0,4003,AV. Cali,Carrera 86,Calle 80,1,1,0,0,INTERMEDIA,NO,4.702500,-74.100253
1,7004,Venecia,AutoSur-Av 54,NQS,2,1,0,0,SENCILLA,NO,4.595713,-74.141590
2,4105,Carrera 53,Carrera 53,Calle 80,1,1,0,0,SENCILLA,NO,4.681940,-74.077365
3,7104,Simón Bolívar,Kr 30 CL 63,NQS,2,1,0,0,SENCILLA,NO,4.657063,-74.077754
4,4000,Portal de la 80,Carrera 96,Calle 80,0,0,1,353,CABECERA,SI,4.709523,-74.110525


In [31]:
estaciones.to_parquet('../data/02_processed/estaciones.parquet')

## Movilidad
La idea con estos datos es sacar a que puntos se estan moviendo las personas. Con esto, podemos estimar que tan lejos viven las personas de su trabajo.

In [32]:
# cargamos datos de viajes
viajes=pd.read_csv('../data/01_raw/movilidad/ViajesEODH2019.csv', sep=';')

In [33]:
viajes.shape

(134497, 36)

In [35]:
viajes.groupby('p29_id_municipio').p29_id_municipio.count().sort_values().tail()

p29_id_municipio
25473.0      1075
25758.0      1078
25175.0      1529
25754.0      3667
11001.0    109719
Name: p29_id_municipio, dtype: int64

In [36]:
#viajes.info()
viajes.columns

Index(['id_hogar', 'id_persona', 'id_viaje', 'fecha', 'lugar_origen',
       'zat_origen', 'p17_Id_motivo_viaje', 'p17_otro_motivo',
       'hora_inicio_viaje', 'p28_lugar_destino', 'zat_destino',
       'p29_id_municipio', 'p30_camino_cuadras', 'p30_camino_minutos',
       'p31_hora_llegada', 'p32_lunes', 'p32_martes', 'p32_miercoles',
       'p32_jueves', 'p32_viernes', 'p32_sabado', 'p32_domingo',
       'p32_ocasional', 'p33_aplicacion_antes_viaje',
       'p33_cual_aplicacion_antes_viaje', 'p34_aplicacion_durante_viaje',
       'p34_cual_aplicacion_durante_viaje', 'p35_otro_desplazamiento',
       'p36_hora_salida', 'f_exp', 'mun_origen', 'mun_destino', 'utam_origen',
       'utam_destino', 'modo_principal', 'modo_principal_desagregado'],
      dtype='object')

In [37]:
# keep only data well likely use
viajes = (
    viajes.
    loc[lambda x:x.p29_id_municipio==11001]
    [['id_hogar', 'id_persona', 'id_viaje', 'fecha', 'lugar_origen',
       'zat_origen', 'p17_Id_motivo_viaje',
       'hora_inicio_viaje', 'p28_lugar_destino',
       'p30_camino_cuadras', 'p30_camino_minutos',
       'p31_hora_llegada', 'p32_lunes', 'p32_martes', 'p32_miercoles',
       'p32_jueves', 'p32_viernes', 'p32_sabado', 'p32_domingo',
       'p32_ocasional', 'p35_otro_desplazamiento',
       'p36_hora_salida', 'f_exp', 'utam_origen',
       'utam_destino', 'modo_principal', 'modo_principal_desagregado']])

Hacemos algo de exploracion para ver que filtros son relevantes, esta exploracion fue complementada en excel.

los viajes con `p17==1` indican trabajo, tambien incluimos solamente viajes frecuentes

In [50]:
# los viajes frecuentes van a corresponder probablemente a trabajadores
viajes_frecuentes = viajes.loc[lambda x: (x['p32_ocasional'].isnull()) & (x['p17_Id_motivo_viaje']==1)]

In [51]:
viajes_frecuentes.shape

(13255, 27)

In [52]:
viajes_frecuentes.columns

Index(['id_hogar', 'id_persona', 'id_viaje', 'fecha', 'lugar_origen',
       'zat_origen', 'p17_Id_motivo_viaje', 'hora_inicio_viaje',
       'p28_lugar_destino', 'p30_camino_cuadras', 'p30_camino_minutos',
       'p31_hora_llegada', 'p32_lunes', 'p32_martes', 'p32_miercoles',
       'p32_jueves', 'p32_viernes', 'p32_sabado', 'p32_domingo',
       'p32_ocasional', 'p35_otro_desplazamiento', 'p36_hora_salida', 'f_exp',
       'utam_origen', 'utam_destino', 'modo_principal',
       'modo_principal_desagregado'],
      dtype='object')

Calculamos el volumen de viajes agregando sobre todos los viajes frecuentes

In [53]:
# TODO cruzar con las personas y obtener solo el segmento que trabajen en salud y relacionados
# El id de la actividad economica de salud es el 17

In [54]:
viajes_vol = (viajes_frecuentes.
     groupby(['utam_origen', 'utam_destino']).
     agg({'f_exp': 'sum'}).
     reset_index().
     rename(columns={'f_exp': 'viajeros'})
)

In [55]:
viajes_vol.to_parquet('../data/02_processed/viajes_vol.parquet')
viajes_vol.to_csv('../data/02_processed/viajes_vol.csv')

In [56]:
viajes_vol.shape

(5934, 3)

In [57]:
viajes_vol.head()

,utam_origen,utam_destino,viajeros
0,UPR1,UTAM20,23.784754
1,UPR1,UTAM27,120.316550
2,UPR1,UTAM33,113.451124
3,UPR1,UTAM38,19.843237
4,UPR1,UTAM575,285.950439


In [ ]:
# datos procesados de centroides en notebook :UTAM